# Predictive modeling notebook

This notebook contains the modeling approach using topic distributions.

## Topic distributions and sentiment

In [ ]:
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
import shap
import xgboost as xgb
from scipy.stats import fisher_exact
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     cross_val_predict)
from sklearn.svm import SVC
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller, kpss

Import data and setup to match

In [ ]:
inflation = pd.read_csv("../dataset/inflation_clean.csv")
unemp = pd.read_csv("../dataset/unemp_clean.csv")

In [ ]:
inflation.rename(columns={"Unnamed: 0": "date"}, inplace=True)
unemp.rename(columns={"Unnamed: 0": "date"}, inplace=True)

In [ ]:
unemp["date"] = [datetime.strptime(date_str, "%YM%m") for date_str in unemp["date"]]
inflation["date"] = [
    datetime.strptime(date_str, "%YM%m") for date_str in inflation["date"]
]

In [ ]:
df_topic_q = pd.read_csv("../dataset/topic_q_downsampled.csv", index_col="date")
df_topic_a = pd.read_csv("../dataset/topic_a_downsampled.csv", index_col="date")
df_sent = pd.read_csv("../dataset/sent_downsampled.csv", index_col="date")

In [ ]:
df_sent_topic = pd.merge(
    df_topic_a.iloc[:, 1:], df_sent["answers"], left_on="date", right_on="date"
)

In [ ]:
df_sent_topic = pd.concat(
    [df_topic_q.iloc[:, 1:], df_topic_a.iloc[:, 1:], df_sent[["questions", "answers"]]],
    axis=1,
)

In [ ]:
df_sent_topic.columns = [
    "topic_1_q",
    "topic_2_q",
    "topic_3_q",
    "topic_4_q",
    "topic_5_q",
    "topic_1_a",
    "topic_2_a",
    "topic_3_a",
    "topic_4_a",
    "topic_5_a",
    "sent_q",
    "sent_a",
]

In [ ]:
def full_log_likelihood(w, X, y):
    score = np.dot(X, w).reshape(1, X.shape[0])
    return np.sum(-np.log(1 + np.exp(score))) + np.sum(y * score)


def null_log_likelihood(w, X, y):
    z = np.array(
        [w if i == 0 else 0.0 for i, w in enumerate(w.reshape(1, X.shape[1])[0])]
    ).reshape(X.shape[1], 1)
    score = np.dot(X, z).reshape(1, X.shape[0])
    return np.sum(-np.log(1 + np.exp(score))) + np.sum(y * score)


def mcfadden_rsquare(w, X, y):
    return 1.0 - (full_log_likelihood(w, X, y) / null_log_likelihood(w, X, y))

### Stationarity

In [ ]:
# Augmented Dickey-Fuller Test (ADF Test)/unit root test
def adf_test(ts, signif=0.05):
    dftest = adfuller(ts, autolag="AIC")
    adf = pd.Series(
        dftest[0:4], index=["Test Statistic", "p-value", "# Lags", "# Observations"]
    )
    for key, value in dftest[4].items():
        adf["Critical Value (%s)" % key] = value

    p = adf["p-value"]
    if p > signif:
        print(f"Series is Non-Stationary")


# KPSS
def kpss_test(ts):
    kpsstest = kpss(ts, regression="c")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    if kpss_output["p-value"] > 0.05:
        print("Stationary")

In [ ]:
def optimise_pls_cv(X, y, n_comp):
    # Define PLS object
    pls = PLSRegression(n_components=n_comp)

    # Cross-validation
    y_cv = cross_val_predict(pls, X, y, cv=10)

    # Calculate scores
    r2 = r2_score(y, y_cv)
    mse = mean_squared_error(y, y_cv)
    rpd = y.std() / np.sqrt(mse)

    return (y_cv, r2, mse, rpd)

# Prediction for unemployment

Stationarity measures

In [ ]:
for col in df_sent_topic.columns:
    print(col)
    adf_test(df_sent_topic[col])

In [ ]:
for col in df_sent_topic.columns:
    print(col)
    kpss_test(df_sent_topic[col])

In [ ]:
for col in df_sent_topic.columns:
    df_sent_topic[col] = df_sent_topic[col] - df_sent_topic[col].shift(1)

Match X and y to have same length and for X to correspond to y in next month

In [ ]:
X_unemp = df_sent_topic[1:-4]
y_unemp = unemp[8:-2]
y_unemp["binary"] = [1 if x > 0 else 0 for x in y_unemp["Delta"]]

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = X_unemp.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_unemp.values, i) for i in range(len(X_unemp.columns))
]

vif_data

In [ ]:
pls_results = []
for n_comp in range(2, len(X_unemp.columns)):
    out = optimise_pls_cv(X_unemp, y_unemp["binary"], n_comp=n_comp)
    pls_results.append(out)

In [ ]:
mse_list = [result[2] for result in pls_results]
r2_list = [result[1] for result in pls_results]

print(mse_list)
r2_list

In [ ]:
X_train = X_unemp.iloc[:137]
X_test = X_unemp.iloc[137:]
y_train = y_unemp["binary"].iloc[:137]
y_test = y_unemp["binary"].iloc[137:]

In [ ]:
pls = PLSRegression(n_components=2)
pls.fit(X_train, y_train)

In [ ]:
X_train = pls.transform(X_train)
X_test = pls.transform(X_test)

## Logistic Regression

In [ ]:
log_mod = LogisticRegression()

log_mod.fit(X_train, y_train)

In [ ]:
# Hyperparameter tuning
grid = {
    # 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    "solver": ["newton-cg"],
    "penalty": ["none", "l2"],
    "C": [100, 10, 1.0, 0.1, 0.01],
}

clf_unemp = GridSearchCV(log_mod, grid, verbose=1, n_jobs=2)

clf_unemp.fit(X_train, y_train)
log_mod = clf_unemp.best_estimator_
log_mod.fit(X_train, y_train)

In [ ]:
y_pred = clf_unemp.predict(X_test)

results = classification_report(y_test, y_pred)

print(results)
print(accuracy_score(y_test, y_pred))

y_test.mean()

In [ ]:
w = np.array(log_mod.coef_).transpose()
mcfadden_rsquare(w, X_test, y_test.to_numpy())

In [ ]:
w

## Support Vector Classifier

In [ ]:
svc_mod = SVC()

svc_mod.fit(X_train, y_train)

In [ ]:
# Hyperparameter tuning
# Start with kernel
grid = {
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    # "kernel": ['poly'],
    # 'C': [0.1, 1, 10, 100, 1000],
    # 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]
}

clf_unemp = GridSearchCV(svc_mod, grid, verbose=1, n_jobs=-1)

clf_unemp.fit(X_train, y_train)

svc_mod = clf_unemp.best_estimator_
svc_mod.fit(X_train, y_train)

In [ ]:
y_pred = clf_unemp.predict(X_test)

results = classification_report(y_test, y_pred)

print(results)
print(accuracy_score(y_test, y_pred))

y_test.mean()

In [ ]:
# Fits the explainer
explainer = shap.Explainer(svc_mod.predict, X_test)
# Calculates the SHAP values - It takes some time
shap_values = explainer(X_test)

In [ ]:
np.mean(abs(shap_values.values[:, 1]))

## Random Forest Classifier

In [ ]:
rfc_mod = RandomForestClassifier(random_state=0)

rfc_mod.fit(X_train, y_train)

In [ ]:
# Hyperparameter tuning
grid = {
    "n_estimators": [int(x) for x in np.linspace(start=50, stop=500, num=10)],
    "max_depth": [2, 4, 6, 8, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["auto", "sqrt"],
    "bootstrap": [True, False],
}

clf_unemp = GridSearchCV(rfc_mod, grid, verbose=1, n_jobs=4)

clf_unemp.fit(X_train, y_train)

In [ ]:
y_pred = clf_unemp.predict(X_test)

results = classification_report(y_test, y_pred)
print(accuracy_score(y_test, y_pred))

print(results)

y_test.mean()

## XGBoost

In [ ]:
xgb_mod = xgb.XGBClassifier(
    random_state=0,
    use_label_encoder=False,
    eval_metric="logloss",
    tree_method="gpu_hist",
    verbosity=2,
)

xgb_mod.fit(X_train, y_train)

In [ ]:
# Hyperparameter tuning
grid = {
    "eta": [0.1, 0.2, 0.3],
    "min_child_weight": [5, 10],
    "gamma": [0, 1.0, 10],
    "subsample": np.arange(0.5, 1, 0.1),
    "colsample_bytree": np.arange(0.5, 1, 0.1),
    "max_depth": np.arange(3, 10, 2),
    "scale_pos_weight": [0.5, 1, 2],
    "reg_alpha": [0, 1, 10.0, 100.0],
    "reg_lambda": [0, 1, 10.0, 100.0],
}

# clf_unemp = GridSearchCV(xgb_mod, grid, verbose=2, n_jobs=-1)

clf_unemp = RandomizedSearchCV(xgb_mod, grid, verbose=2, n_jobs=-1, n_iter=10000)

model = clf_unemp.fit(X_train, y_train)

with open("../models/xgb_all_unemp.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
y_pred = clf_unemp.predict(X_test)

results = classification_report(y_test, y_pred)
print(accuracy_score(y_test, y_pred))

print(results)

y_test.mean()

In [ ]:
model

## Unemployment Fisher's exact

Below we test whether the accuracy of the model above is beteter during times of high/low volatility.

In [ ]:
y_pred_log = log_mod.predict(X_test)
y_pred_svc = svc_mod.predict(X_test)

In [ ]:
y_test = y_test.tolist()

In [ ]:
y_correct_log = [1 if pred == y_test[i] else 0 for i, pred in enumerate(y_pred_log)]
y_correct_svc = [1 if pred == y_test[i] else 0 for i, pred in enumerate(y_pred_svc)]

In [ ]:
volatility = [0] * (59 - 24) + [1] * 24

In [ ]:
log_con = pd.crosstab(y_correct_log, volatility).to_numpy()
svc_con = pd.crosstab(y_correct_svc, volatility).to_numpy()

In [ ]:
fisher_exact(log_con)

In [ ]:
fisher_exact(svc_con)

# Prediction for inflation

In [ ]:
X_inflation = df_sent_topic[1:-1]
y_inflation = inflation[8:]
y_inflation["binary"] = [1 if x > 0 else 0 for x in y_inflation["Delta"]]

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = X_inflation.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_inflation.values, i)
    for i in range(len(X_inflation.columns))
]

vif_data

In [ ]:
X_train = X_inflation.iloc[:139, 1:]
X_test = X_inflation.iloc[139:, 1:]
y_train = y_inflation["binary"].iloc[:139]
y_test = y_inflation["binary"].iloc[139:]

In [ ]:
pls_results = []
for n_comp in range(2, len(X_inflation.columns)):
    out = optimise_pls_cv(X_inflation, y_inflation["binary"], n_comp=n_comp)
    pls_results.append(out)

mse_list = [result[2] for result in pls_results]
r2_list = [result[1] for result in pls_results]

print(mse_list)
r2_list

In [ ]:
pls = PLSRegression(n_components=4)
pls.fit(X_train, y_train)

In [ ]:
X_train = pls.transform(X_train)
X_test = pls.transform(X_test)

## Logistic Regression

In [ ]:
log_mod = LogisticRegression()

log_mod.fit(X_train, y_train)

In [ ]:
# Hyperparameter tuning
grid = {
    # 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    "solver": ["newton-cg"],
    "penalty": ["none", "l2"],
    "C": [100, 10, 1.0, 0.1, 0.01],
}

clf_inf = GridSearchCV(log_mod, grid, verbose=1, n_jobs=-1)

clf_inf.fit(X_train, y_train)
clf_inf.best_estimator_

In [ ]:
y_pred = clf_inf.predict(X_test)

results = classification_report(y_test, y_pred)

print(results)
print(accuracy_score(y_test, y_pred))

y_test.mean()

## Support Vector Classifier

In [ ]:
svc_mod = SVC()

svc_mod.fit(X_train, y_train)

In [ ]:
# Hyperparameter tuning
# Start with kernel
grid = {
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    # "kernel": ['poly'],
    # 'C': [0.1, 1, 10, 100, 1000],
    # 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]
}

clf_inf = GridSearchCV(svc_mod, grid, verbose=1, n_jobs=-1)

clf_inf.fit(X_train, y_train)
clf_inf.best_estimator_

In [ ]:
y_pred = clf_inf.predict(X_test)

results = classification_report(y_test, y_pred)

print(results)
print(accuracy_score(y_test, y_pred))

y_test.mean()

## Random Forest Classifier

In [ ]:
rfc_mod = RandomForestClassifier(random_state=0)

rfc_mod.fit(X_train, y_train)

In [ ]:
# Hyperparameter tuning
grid = {
    "n_estimators": [int(x) for x in np.linspace(start=50, stop=500, num=10)],
    "max_depth": [2, 4, 6, 8, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["auto", "sqrt"],
    "bootstrap": [True, False],
}

clf_inf = GridSearchCV(rfc_mod, grid, verbose=1, n_jobs=6)

clf_inf.fit(X_train, y_train)

In [ ]:
y_pred = clf_inf.predict(X_test)

results = classification_report(y_test, y_pred)

print(results)
print(accuracy_score(y_test, y_pred))

y_test.mean()

## XGBoost

In [ ]:
xgb_mod = xgb.XGBClassifier(
    random_state=0,
    use_label_encoder=False,
    eval_metric="logloss",
    tree_method="gpu_hist",
)

xgb_mod.fit(X_train, y_train)

In [ ]:
# Hyperparameter tuning
grid = {
    "eta": [0.1, 0.2, 0.3],
    "min_child_weight": [5, 10],
    "gamma": [0, 1.0, 10],
    "subsample": np.arange(0.5, 1, 0.1),
    "colsample_bytree": np.arange(0.5, 1, 0.1),
    "max_depth": np.arange(3, 10, 2),
    "scale_pos_weight": [0.5, 1, 2],
    "reg_alpha": [0, 1, 10.0, 100.0],
    "reg_lambda": [0, 1, 10.0, 100.0],
}

clf_inf = GridSearchCV(xgb_mod, grid, verbose=2, n_jobs=4)

# clf_inf = RandomizedSearchCV(xgb_mod, grid, verbose=1, n_jobs=4,
#                              random_state=0, n_iter=10000)

model = clf_inf.fit(X_train, y_train)

# with open('../models/xgb_all_inf.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [ ]:
y_pred = clf_inf.predict(X_test)

results = classification_report(y_test, y_pred)

print(results)
print(accuracy_score(y_test, y_pred))

y_test.mean()